In [4]:
import pydicom
import os

In [5]:
Dataset='/root/Data/MRTData/MRT/DCM/JR003254-MRT'

In [26]:
for file in os.listdir(DataDir):
    ds = pydicom.filereader.dcmread(os.path.join(DataDir,file))
    if  ds.Modality == 'T1' and ds.Modality=='MR':
        print(ds.Modality)
    
#ds = pydicom.filereader.dcmread('/root/Data/MRTData/MRT/DCM/JR003021-MRT/IMAGE/DCM/12929897/I0000000')

In [9]:
print(ds.Modality)

MR


In [5]:
ds = pydicom.filereader.dcmread('/root/Data/MRTData/MRT/DCM/JR003021-MRT/IMAGE/DCM/12929897/I0000033')

In [7]:
print(ds.Modality)

MR


In [28]:
 ds = pydicom.filereader.dcmread('/root/Data/MRTData/MRT/DCM/JR003021-MRT/IMAGE/DCM/12929897/I0000004')

In [29]:
print(ds)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 194
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.46.670589.11.8690.5.0.5232.2019021820273170899
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.410.200010.99.3.5
(0002, 0013) Implementation Version Name         SH: 'INF_4.5'
(0002, 0016) Source Application Entity Title     AE: 'MRT1HOST'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M_FFE', 'M', 'FFE']
(0008, 0012) Instance Creation Date              DA: '20190218'
(0008, 0013) Instance Creation Time              TM: '205247'
(0008, 0014) Instance Creator UID

In [4]:
import os
import pydicom # pydicom is using the gdcm package for decompression

def clean_text(string):
    # clean and standardize text descriptions, which makes searching files easier
    forbidden_symbols = ["*", ".", ",", "\"", "\\", "/", "|", "[", "]", ":", ";", " "]
    for symbol in forbidden_symbols:
        string = string.replace(symbol, "_") # replace everything with an underscore
    return string.lower()  
   
# user specified parameters
src = "/root/Data/MRTData/MRT"
dst = "/root/code/thesis/codeFolder/LatestDataInUse/TryMRTSequences"

print('reading file list...')
unsortedList = []
for root, dirs, files in os.walk(src):
    for file in files:        
        unsortedList.append(os.path.join(root, file))

print('%s files found.' % len(unsortedList))
       
for dicom_loc in unsortedList:
    # read the file
    ds = pydicom.read_file(dicom_loc, force=True)
   
    # get patient, study, and series information
    patientID = clean_text(ds.get("PatientID", "NA"))
    studyDate = clean_text(ds.get("StudyDate", "NA"))
    studyDescription = clean_text(ds.get("StudyDescription", "NA"))
    seriesDescription = clean_text(ds.get("SeriesDescription", "NA"))
   
    # generate new, standardized file name
    modality = ds.get("Modality","NA")
    studyInstanceUID = ds.get("StudyInstanceUID","NA")
    seriesInstanceUID = ds.get("SeriesInstanceUID","NA")
    instanceNumber = str(ds.get("InstanceNumber","0"))
    fileName = modality + "." + seriesInstanceUID + "." + instanceNumber + ".dcm"
    #fileName =  instanceNumber + ".dcm"
       
    # uncompress files (using the gdcm package)
    try:
        ds.decompress()
    except:
        print('an instance in file %s - %s - %s - %s" could not be decompressed. exiting.' % (patientID, studyDate, studyDescription, seriesDescription ))
   
    # save files to a 4-tier nested folder structure
    if not os.path.exists(os.path.join(dst, patientID)):
        os.makedirs(os.path.join(dst, patientID))
   
    if not os.path.exists(os.path.join(dst, patientID, studyDate)):
        os.makedirs(os.path.join(dst, patientID, studyDate))
       
    if not os.path.exists(os.path.join(dst, patientID, studyDate, studyDescription)):
        os.makedirs(os.path.join(dst, patientID, studyDate, studyDescription))
       
    if not os.path.exists(os.path.join(dst, patientID, studyDate, studyDescription, seriesDescription)):
        os.makedirs(os.path.join(dst, patientID, studyDate, studyDescription, seriesDescription))
        print('Saving out file: %s - %s - %s - %s.' % (patientID, studyDate, studyDescription, seriesDescription ))
    print(fileName)
       
    ds.save_as(os.path.join(dst, patientID, studyDate, studyDescription, seriesDescription, fileName))

print('done.')

reading file list...
29051 files found.
Saving out file: jr003643-mrt - 20180305 - mrt_der_leber - ddwi_pace_neu_sense.
MR.1.3.6.1.4.1.5962.99.1.4014633455.1294938462.1666167009775.784.0.340.dcm
Saving out file: jr003643-mrt - 20180305 - mrt_der_leber - e-thrive_bh.
MR.1.3.6.1.4.1.5962.99.1.4014633455.1294938462.1666167009775.164.0.96.dcm
MR.1.3.6.1.4.1.5962.99.1.4014633455.1294938462.1666167009775.164.0.4.dcm
Saving out file: jr003643-mrt - 20180305 - mrt_der_leber - dwi_pace_neu.
MR.1.3.6.1.4.1.5962.99.1.4014633455.1294938462.1666167009775.693.0.128.dcm
Saving out file: jr003643-mrt - 20180305 - mrt_der_leber - se-thrive_bh_sense.
MR.1.3.6.1.4.1.5962.99.1.4014633455.1294938462.1666167009775.425.0.150.dcm
MR.1.3.6.1.4.1.5962.99.1.4014633455.1294938462.1666167009775.988.0.47.dcm
MR.1.3.6.1.4.1.5962.99.1.4014633455.1294938462.1666167009775.425.0.154.dcm
MR.1.3.6.1.4.1.5962.99.1.4014633455.1294938462.1666167009775.693.0.244.dcm
MR.1.3.6.1.4.1.5962.99.1.4014633455.1294938462.1666167009775

In [1]:
Dataset='/root/Data/CTMRTExcelLat/CTSequences'

In [2]:
from collections import defaultdict
import os


my_dict = defaultdict(list)

In [6]:
#pat={}
for patient in os.listdir(Dataset):
    for folder in os.listdir(os.path.join(Dataset,patient)):
        for folder1 in os.listdir(os.path.join(Dataset,os.path.join(patient,folder))):
            for subFolder in os.listdir(os.path.join(Dataset,os.path.join(patient,os.path.join(folder,folder1)))):
                my_dict[str(patient)].append(subFolder)
                #print(subFolder)
                

In [7]:
my_dict

defaultdict(list,
            {'jr003803-ct': ['body_5_000_ce', '2_0', 'body_5_0_ce'],
             'jr003093-ct': ['abd__ven_phase__5_0__i30f__3',
              'lun__ven_phase__5_0__i70f__3',
              'tho__ven_phase__5_0__i31f__3',
              'art__phase__5_0__i30f__3',
              'premonitoring__10_0__b30s',
              'monitoring__10_0__b30s',
              'p-v__phase__5_0__i30f__3',
              'topogramm__0_6__t20f'],
             'jr003376-ct': ['body_2_0',
              'body_5_000_ce',
              'body_5_0',
              '2_0',
              'body_5_0_ce',
              'body_5_000'],
             'jr009062-ct': ['body_2_0',
              '2_0',
              'lung_5_0_ce',
              'body_5_000_ce_sagittal',
              'body_5_0_ce'],
             '298-321_ct': ['body_2_0',
              'body_5_000_ce',
              '2_0',
              'body_1_0_ce',
              'lung_5_0_ce',
              'body_5_0_ce',
              'lung_1_0_ce'],
       

In [18]:
for key in my_dict.keys():
    print (key)
    print(my_dict[key])

jr003093-mrt
['t1fs_3d_hbp_cor', 'ddwi_sense', 'se-thrive_bh_sense', 'dwi', 't1fs_3d_hbp_tra', 'survey_breath_hold', 'btfe_cor', 't2w_ssh_tra', 't1fs_dyn', 't2w_ssh_cor', 't1fs_2d_hbp_tra', 'btfe_tra', 't1w_ip-op_tra', 't2fs_tra']
jr003809-mrt
['se-thrive_bh_sense', 's3t1_wats_neu', 'survey_breath_hold', 'e-thrive_bh', 'e-thrive_', 't2_hr__range_slab', 'ssh_te80_cor', 't2_hr_spir_range']
jr003529-mrt
['se-thrive_bh_sense', 's3t1_wats_neu', 'survey_breath_hold', 'e-thrive_bh', 'e-thrive_', 't2_hr__range_slab', 'ssh_te80_cor', 't2_hr_spir_range']
jr005882-mrt
['se-thrive_bh_sense', 's3t1_wats_20min', 'survey_breath_hold', 'e-thrive_bh', 'e-thrive_', 'ssh_te80_cor', 't2_hr_spir_range', 'e-thr__b_hcc']
jr003642-mrt
['2d_bolustrak', 'ssh_drive', 'se-thrive_bh_sense', 'ddwi_pace_neu_sense', 'btfe_110dg_trigtrac', 's3t1_wats_neu', 'dwi_pace_neu', 'survey_breath_hold', 'e-thrive_bh', 'dual_neu_1', 'btfe_110dg_trigtrac_sense', 'e-thrive_', 'ssh_te80_cor', 't2_hr_spir_range']
jr005691-mrt
['ssh_

In [8]:
my_dict.keys()

dict_keys(['jr003803-ct', 'jr003093-ct', 'jr003376-ct', 'jr009062-ct', '298-321_ct', 'jr003241-ct', 'jr009082-ct', 'jr005691-ct', 'jr003525-ct', 'jr009039-ct', '298-353_ct', 'jr009032-ct', 'jr005882-ct', 'jr005990-ct', 'jr003534-ct', 'jr003642-ct', 'jr003021-ct', 'jr003912-ct', '298-131_ct', 'jr005667-ct'])

In [13]:
flipped = {}
for key, value in my_dict.items():
    if value not in flipped:
        flipped[value] = [key]
    else:
        flipped[value].append(key)
 
# printing result
print("final_dictionary", str(flipped))

TypeError: unhashable type: 'list'

In [14]:
k_v_exchanged = {}

for key, value in my_dict.items():
   if value not in k_v_exchanged:
      k_v_exchanged[value] = [key]
   else:
      k_v_exchanged[value].append(key)

# Result
print("New Dictionary:", k_v_exchanged)

TypeError: unhashable type: 'list'

In [3]:
pat={}
for patient in os.listdir(Dataset):
    print(patient)
    for folder in os.listdir(os.path.join(Dataset,patient)):
        for folder1 in os.listdir(os.path.join(Dataset,os.path.join(patient,folder))):
            for subFolder in os.listdir(os.path.join(Dataset,os.path.join(patient,os.path.join(folder,folder1)))):
                my_dict[str(subFolder)].append(patient)
                #print(subFolder)

jr003803-ct
jr003093-ct
jr003376-ct
jr009062-ct
298-321_ct
jr003241-ct
jr009082-ct
jr005691-ct
jr003525-ct
jr009039-ct
298-353_ct
jr009032-ct
jr005882-ct
jr005990-ct
jr003534-ct
jr003642-ct
jr003021-ct
jr003912-ct
298-131_ct
jr005667-ct


In [4]:
my_dict


defaultdict(list,
            {'body_5_000_ce': ['jr003803-ct',
              'jr003376-ct',
              '298-321_ct',
              'jr003912-ct'],
             '2_0': ['jr003803-ct',
              'jr003376-ct',
              'jr009062-ct',
              '298-321_ct',
              'jr009082-ct',
              'jr005691-ct',
              'jr003525-ct',
              'jr009032-ct',
              'jr005882-ct',
              'jr005990-ct',
              'jr003534-ct',
              'jr003642-ct',
              'jr003912-ct',
              '298-131_ct',
              'jr005667-ct'],
             'body_5_0_ce': ['jr003803-ct',
              'jr003376-ct',
              'jr009062-ct',
              '298-321_ct',
              'jr009082-ct',
              'jr005691-ct',
              'jr003525-ct',
              'jr009032-ct',
              'jr005882-ct',
              'jr005990-ct',
              'jr003534-ct',
              'jr003642-ct',
              'jr003912-ct',
              '29

In [7]:
with open('/root/Data/CTMRTExcelLat/sequences.txt', 'w') as file:
     file.write(json.dumps(my_dict))

In [6]:
import json